# Types of SK Plugin Functions

First, as a reminder, Semantic Kernel allows you to define Plugins (formerly known as Skills) which encapsulate one or more Functions. These functions are where you will engineer your prompts (as well as insert and access variables). There are three types of plugin functions supported:

- *Inline*: inline functions are written in your chosen native language. You create the prompt, any variables, and configuration options in your desired language.
- *Semantic*: semantic functions are written in something closer to natural language. While this may initially seem too basic or limited, this non-traditional code can often be a very powerful approach in this A.I. world. ("English is the new programming language.")
- *Native*: like inline functions, native functions are written in your chosen native language, but they allow you to wrap any code logic as a function to be used within the kernel.

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.0.0-beta8"
#!import ../config/SettingsHelper.cs
using Microsoft.SemanticKernel;

// the following is an abbreviated version of the code in Tutorial0 that reads from the settings file and sets up the kernel
MySettings settings = Settings.LoadFromFile();
KernelBuilder builder = new();
if (settings.Type == "azure")
{
    if (!string.IsNullOrWhiteSpace(settings.AzureOpenAI.ChatDeployment))
        builder.WithAzureOpenAIChatCompletionService(settings.AzureOpenAI.ChatDeployment, settings.AzureOpenAI.Endpoint, settings.AzureOpenAI.ApiKey);
}
else
    builder.WithOpenAIChatCompletionService(settings.OpenAI.Model, settings.OpenAI.ApiKey, settings.OpenAI.OrgId);
IKernel kernel = builder.Build();

## Inline Function

You can create a function alongside the rest of your code. These inline functions are very easy to implement and may allow for some more complex prompt construstion, but it potentially sacrifices some reusability and maintainability.

Run the following code box just once. If you want to run it again, run the code box at the top of this page first otherwise you will keep adding copies of the same function to the kernel. 

In [3]:
using Microsoft.SemanticKernel.Orchestration;

// 1. define your inline function
string functionDefinition = "Write the following words in reverse order: {{$input}}";

// 2. convert the prompt template above into a function and register it with the kernel
ISKFunction reverseFunction = kernel.CreateSemanticFunction(functionDefinition);


The code above shows another example of an inline function - as you've seen in the previous tutorials.

1. This is just a simple prompt template string again. And as you've seen previously, we are just using the pre-defined input variable for the Semantic Kernel to swap out.
2. Then we let Semantic Kernel convert that prompt template into a semantic function and register it with our kernel instance for use later.

In [ ]:
// user interaction - feel free to modify this code box and re-run as desired
// insert your prompt in your inline function
string prompt = "If you know the enemy and know yourself, you need not fear the result of a hundred battles. If you know yourself but not the enemy, for every victory gained you will also suffer a defeat. If you know neither the enemy nor yourself, you will succumb in every battle.";

// invoke the function to pass the prompt as input to the function - and then the function to OpenAI
FunctionResult result = await reverseFunction.InvokeAsync(prompt, kernel)
    ?? throw new ApplicationException("Something went wrong, please check your settings.");

Console.WriteLine(result.GetValue<string>()!);

We first define our inline function using a standard string. The strange-looking {{$input}} is a variable - a placeholder for the rest of the text we'll provide before sending it to OpenAI.

The second line of code uses the kernel's CreateSemanticFunction method to turn our string into a real Semantic Kernel function. The function acts like a template that we customize with additional information - in this case a quote from a world-famous philosopher and strategist.

# Semantic Functions

Semantic functions are especially interesting. Up to this point, we defined our functions using traditional C# code and there's a lot of great use cases for doing so. But in a world of A.I., why can't we just use a more natural language to define our function? A semantic function allows for exactly that!

Semantic functions are defined by two files - and a particular folder structure. The top-level folder is the plugin (formerly known as a skill) and it contains a variety of related functions. Each function is defined by two files: A config.json file defines the parameters to pass to OpenAI's API and the skprompt.txt file defines the actual prompt and any variables.

Before you run the code below, open each of the two files in the Plugins/TranslatePlugin/PigLatinFunction folder.
- The most interesting file is the skprompt.txt file which is our function definition. The {{$input}} is a built-in variable that we'll replace with the rest of our prompt. (We'll discuss supporting multiple parameters in our Context Variables tutorial.)
- We'll dive into the config.json options in another tutorial, but it's good to at least see what that file contains for now.

In [ ]:
using System.IO;

// load the plugin from your folder structure
string pluginDirectory = Path.Combine(Directory.GetCurrentDirectory(), "Plugins");
var translatePlugin = kernel.ImportSemanticSkillFromDirectory(pluginDirectory, "TranslatePlugin");

string prompt = "Did you ever dance with the devil in the pale moonlight?";

// run the specified function within a specific plugin
var result = await kernel.RunAsync(prompt, translatePlugin["PigLatinFunction"]);

Console.WriteLine(result);

In the above example, the first two lines load the Semantic Kernel up with the TranslatePlugin in our Plugins folder. We then pass the prompt into our PigLatinFunction. The RunAsync method of the kernel inserts our prompt into the function and sends the entire thing to OpenAI to calculate the completion. (I think the character associated with this quote would appreciate our work here. :D )

# Native Functions

Like inline functions, native functions are written in code. Since it's "just code", native functions provide the same opportunity to do anything you can do in code (basically everything! :) )

*You'll learn more about native function capabilities in the prompt chaining tutorial.*

Before you run the following code, find the OldeEnglishPlugin.cs file in the Plugins folder. Open it up and look at the very simple code to see if you can predict what it's going to do.

In [ ]:
#!import Plugins/OldeEnglishPlugin.cs

var oePlugin = kernel.ImportSkill(new OldeEnglishPlugin());

var context = kernel.RunAsync(
    "Death cannot stop true love. All it can do is delay it for a while.", 
    oePlugin["Translate"]);

Console.WriteLine(context.Result);

Hmmm...that may not have been the output we expected, but it is behaving correctly. Unlike the semantic and inline functions we saw above, native functions aren't intrinsically wired into OpenAI. Native functions allow you to take any custom code and treat it as a plugin that can be used within the kernel. Each method in the class can be designated as a function by decorating the method with the SKFunction attribute.

If you want to incorporate AI into your function and need to work in code, consider inline functions first as they are simple, but if you want to add AI into your native function (or you really just want to see the Olde English interpretation of the quote above :) ), check out the following example.

In [ ]:
#!import Plugins/OldeEnglishPlugin2.cs

var oePlugin = kernel.ImportSkill(new OldeEnglishPlugin2());

var context = kernel.RunAsync(
    "Death cannot stop true love. All it can do is delay it for a while.", 
    oePlugin["Translate"]);

Console.WriteLine(context.Result);

That's a little better! Open the OldeEnglishPlugin2.cs file and compare it to the OldeEnglishPlugin.cs you looked at above. We'll look at different ways to use native functions in other tutorials.

## Exercise

With each of the above functions now associated with the kernel, write code in the box below to invoke each function with your own input prompt and write the output of each to the Console window.

# Exercise

Add your own semantic function to the Tutorial02 folder and test that you get the desired result.
Add your own native function to the Tutorial02 folder and test that you get the desired result.

You've achieved an important milestone because functions and skills/plugins are important building blocks of Semantic Kernel!

You can probably imagine all kinds of things you could start creating! But check out the next tutorial because it's important we start putting all of these building blocks together.